In [301]:
from ib_async import *
import requests, pytz, math
import pandas as pd
import numpy as np
from datetime import datetime, time, timedelta
import pandas_market_calendars as mcal
from zoneinfo import ZoneInfo
import yfinance as yf
from data_and_research import ac
from broker.connection import connect_to_IB
import broker.utilityfunctions as uf

In [302]:
last_full_trading_day = uf.get_last_full_trading_day()
last_full_trading_day

datetime.date(2024, 10, 9)

In [303]:
current_or_next_trading_day = uf.get_current_or_next_trading_day()
current_or_next_trading_day

datetime.date(2024, 10, 10)

In [21]:
start_date = last_full_trading_day
end_date = current_or_next_trading_day + timedelta(days=4)
earnings = uf.get_earnings(start_date=start_date, end_date=end_date)
earnings.head(10)

,symbol,date,when
0,NAPA,2024-10-07,After market close
1,ACCD,2024-10-08,Before market open
2,IDT,2024-10-08,After market close
3,PEP,2024-10-08,Before market open
4,SAR,2024-10-08,After market close
5,APLD,2024-10-09,After market close
6,AZZ,2024-10-09,After market close
7,BSET,2024-10-09,After market close
8,BYRN,2024-10-09,Before market open
9,ETWO,2024-10-09,After market close


In [304]:
earnings = uf.get_earnings()
earnings

,symbol,date,when
0,APLD,2024-10-09,After market close
1,AZZ,2024-10-09,After market close
2,BSET,2024-10-09,After market close
3,DAL,2024-10-10,Before market open
4,DPZ,2024-10-10,Before market open
5,ETWO,2024-10-09,After market close
6,NEOG,2024-10-10,Before market open
7,RELL,2024-10-10,Before market open
8,THTX,2024-10-10,Before market open
9,TLRY,2024-10-10,Before market open


In [305]:
symbols=earnings['symbol'].tolist()
vol_df = uf.get_vol_data(symbols=symbols, curated=True, include_yf=True)

Looking up vol data for: 2024-10-09


In [306]:
vol_df

,act_symbol,date,hv_current,iv_current,vol_premium,close
2,DPZ,2024-10-09,0.2160,0.4407,2.040278,410.690002
0,AZZ,2024-10-09,0.3258,0.4872,1.495396,76.970001
3,NEOG,2024-10-09,0.4262,0.6163,1.446035,13.706000
1,DAL,2024-10-09,0.3123,0.4301,1.377201,49.709999


In [ ]:
ib = connect_to_IB(clientid=0)

In [ ]:
ib.reqMarketDataType(4)


In [24]:
symbols = vol_df.act_symbol.to_list()


In [308]:
options_df = uf.pea_oss(ib, symbols=symbols, max_dte=50, unfiltered=True,strike_range_percent=0.30)



Error 200, reqId 161: No security definition has been found for the request, contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241011', strike=52.5, right='P', exchange='SMART')
Error 200, reqId 167: No security definition has been found for the request, contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241011', strike=57.5, right='P', exchange='SMART')
Error 200, reqId 172: No security definition has been found for the request, contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241011', strike=62.0, right='P', exchange='SMART')
Error 200, reqId 173: No security definition has been found for the request, contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241011', strike=62.5, right='P', exchange='SMART')
Error 200, reqId 174: No security definition has been found for the request, contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241011', strike=63.0, right='P', exchange='SMART')
Error 200, reqId 175: No security definition 

Unknown contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241011', strike=52.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241011', strike=57.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241011', strike=62.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241011', strike=62.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241011', strike=63.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241011', strike=64.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241018', strike=35.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241018', strike=36.5, right='P', exchange='SMART')
Unknown contract: Option

ValueError: too many values to unpack (expected 2)

In [26]:
options_df

""


In [27]:
import asyncio

In [309]:
async def process_get_filtered_put_options(ib, symbol,min_dte=0, max_dte=60, unfiltered=False,strike_range_percent = 0.25):
    
    # Get the stock contract
    stock = Stock(symbol, 'SMART', 'USD')
    await ib.qualifyContractsAsync(stock)
    
    # Get the stock price
    [ticker] = await ib.reqTickersAsync(stock)

    stock_price = ticker.marketPrice() if ticker.marketPrice() is not None else ticker.close
    
    # Calculate strike range
    lower_strike = stock_price * (1-strike_range_percent)  
    upper_strike = stock_price * (1+strike_range_percent) 
    
    # Get option chains
    chains = await ib.reqSecDefOptParamsAsync(stock.symbol, '', stock.secType, stock.conId)
    
    # Get the chain with exchange 'SMART'
    chain = next((c for c in chains if c.exchange == 'SMART'), None)
    if not chain:
        print(f"No SMART chain found for {symbol}")
        return None, None
    
    # Get current date
    today = datetime.now().date()
    
    # Filter expirations and strikes
    valid_expirations = [exp for exp in chain.expirations 
                         if (datetime.strptime(exp, '%Y%m%d').date() - today).days >= min_dte and (datetime.strptime(exp, '%Y%m%d').date() - today).days <= max_dte]
    valid_strikes = [strike for strike in chain.strikes 
                     if lower_strike <= strike <= upper_strike]
    
    # Create option contracts
    contracts = [Option(symbol, exp, strike, 'P', 'SMART') 
                 for exp in valid_expirations 
                 for strike in valid_strikes]
    
    # Qualify the contracts
    contracts = await ib.qualifyContractsAsync(*contracts)
    
    # Request market data
    tickers = await ib.reqTickersAsync(*contracts)
    
    # Create DataFrame
    data = []
    for ticker in tickers:
        contract = ticker.contract
        expiration = datetime.strptime(contract.lastTradeDateOrContractMonth, '%Y%m%d').date()
        dte = (expiration - today).days
        data.append({
            'Symbol': symbol,
            'StockPrice': stock_price,
            'Strike': contract.strike,
            'Expiration': expiration,
            'DTE': dte,
            'Bid': ticker.bid,
            'BidSize': ticker.bidSize,
            'Ask': ticker.ask,
            'AskSize': ticker.askSize,
            'Contract': contract})
    
    df = pd.DataFrame(data)
    df = df.sort_values(['Expiration', 'Strike'])
    if unfiltered:
        return df
    options_df = df.copy()
    options_df['option_premium'] = options_df['Bid'] / options_df['Strike']
    options_df['annualized_premium'] = options_df['option_premium'] * (365 / options_df['DTE'])
    options_df['safety_margin'] = stock_price - options_df['Strike']
    options_df = options_df[(options_df['option_premium'] > 0.01) & (options_df['annualized_premium'] > 0.1)]
    options_df['safety_margin%'] = options_df['safety_margin'] / options_df['StockPrice']
    
    # Sort the DataFrame after creating all columns
    options_df = options_df.sort_values('safety_margin%', ascending=False)
    
    return options_df.head(10), stock_price

async def get_filtered_put_options(ib, symbols,min_dte = 0,max_dte = 60, unfiltered=False,strike_range_percent=0.25):
    if not isinstance(symbols, list):
        symbols = [symbols]
        
    tasks = [process_get_filtered_put_options(ib, symbol, min_dte, max_dte, unfiltered,strike_range_percent) for symbol in symbols]
    results = await asyncio.gather(*tasks)

    all_options = []
    if not unfiltered:
        for options_df, _ in results:
            if options_df is not None and not options_df.empty:
                all_options.append(options_df)
    else:
        all_options = results
    if all_options:
        return pd.concat(all_options, ignore_index=True)
    else:
        return pd.DataFrame()

In [310]:
# symbols = vol_df.act_symbol.to_list()
df = await get_filtered_put_options(ib, symbols=symbols,unfiltered=True)
df

Unknown contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241011', strike=52.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241011', strike=57.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241011', strike=62.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241018', strike=37.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241018', strike=38.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241018', strike=39.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241018', strike=40.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241018', strike=41.5, right='P', exchange='SMART')
Unknown contract: Option

KeyError: 'Expiration'

Unknown contract: Option(symbol='DPZ', lastTradeDateOrContractMonth='20241018', strike=315.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='DPZ', lastTradeDateOrContractMonth='20241025', strike=392.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='DPZ', lastTradeDateOrContractMonth='20241101', strike=315.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='DPZ', lastTradeDateOrContractMonth='20241101', strike=325.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='DPZ', lastTradeDateOrContractMonth='20241101', strike=335.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='DPZ', lastTradeDateOrContractMonth='20241101', strike=392.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='DPZ', lastTradeDateOrContractMonth='20241101', strike=397.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='DPZ', lastTradeDateOrContractMonth='20241101', strike=402.5, right='P', exchange='SMART')
Unknown contract

""


In [30]:
df.sort_values('safety_margin%', ascending=False)

KeyError: 'safety_margin%'

In [319]:
async def debug_process_get_filtered_put_options(ib, symbol, min_dte=0, max_dte=60, unfiltered=False, min_safety_margin=0.5, max_safety_margin=0.5):
    # Get the stock contract
    stock = Stock(symbol, 'SMART', 'USD')
    await ib.qualifyContractsAsync(stock)
    print(f"Qualified stock contract for {symbol}")

    # Get the stock price
    [ticker] = await ib.reqTickersAsync(stock)
    if type(ticker.marketPrice()) == float and not math.isnan(ticker.marketPrice()):
        stock_price = ticker.marketPrice()
    else:
        stock_price = ticker.close

    # Calculate strike range
    lower_strike = stock_price * (1 - min_safety_margin)
    upper_strike = stock_price * (1 + max_safety_margin)
    print(f"Strike range for {symbol}: {lower_strike} to {upper_strike}")

    # Get option chains
    chains = await ib.reqSecDefOptParamsAsync(stock.symbol, '', stock.secType, stock.conId)
    chain = next((c for c in chains if c.exchange == 'SMART'), None)
    if not chain:
        print(f"No SMART chain found for {symbol}")
        return None, None
    print(f"Option chain found for {symbol}")

    # Get current date
    today = datetime.now().date()

    # Filter expirations and strikes
    valid_expirations = [exp for exp in chain.expirations 
                         if (datetime.strptime(exp, '%Y%m%d').date() - today).days >= min_dte and (datetime.strptime(exp, '%Y%m%d').date() - today).days <= max_dte]
    valid_strikes = [strike for strike in chain.strikes 
                     if lower_strike <= strike <= upper_strike]
    print(f"Valid expirations for {symbol}: {valid_expirations}")
    print(f"Valid strikes for {symbol}: {valid_strikes}")

    # Create option contracts
    contracts = [Option(symbol, exp, strike, 'P', 'SMART') 
                 for exp in valid_expirations 
                 for strike in valid_strikes]
    contracts = await ib.qualifyContractsAsync(*contracts)
    print(f"Qualified option contracts for {symbol}")

    # Request market data
    tickers = await ib.reqTickersAsync(*contracts)
    print(f"Market data received for {symbol}")

    # Create DataFrame
    data = []
    for ticker in tickers:
        contract = ticker.contract
        expiration = datetime.strptime(contract.lastTradeDateOrContractMonth, '%Y%m%d').date()
        dte = (expiration - today).days
        data.append({
            'Symbol': symbol,
            'StockPrice': stock_price,
            'Strike': contract.strike,
            'Expiration': expiration,
            'DTE': dte,
            'Bid': ticker.bid,
            'BidSize': ticker.bidSize,
            'Ask': ticker.ask,
            'AskSize': ticker.askSize,
            'Contract': contract
        })
    df = pd.DataFrame(data)
    if df.empty:
        print(f"No valid options data for {symbol}")
        return None
    print(f"DataFrame created for {symbol}:\n{df.head()}")

    df = df.sort_values(['Expiration', 'Strike'])
    if unfiltered:
        return df
    options_df = df.copy()
    options_df['option_premium'] = options_df['Bid'] / options_df['Strike']
    options_df['annualized_premium'] = options_df['option_premium'] * (365 / options_df['DTE'])
    options_df['safety_margin'] = stock_price - options_df['Strike']
    options_df = options_df[(options_df['option_premium'] > 0.01) & (options_df['annualized_premium'] > 0.1)]
    options_df['safety_margin%'] = options_df['safety_margin'] / options_df['StockPrice']
    options_df = options_df.sort_values('safety_margin%', ascending=False)
    print(f"Filtered options DataFrame for {symbol}:\n{options_df.head()}")

    return options_df.head(10)
     
df = await debug_process_get_filtered_put_options(ib, symbols[4])
df

Qualified stock contract for DPZ
Strike range for DPZ: 204.055 to 612.165
Option chain found for DPZ
Valid expirations for DPZ: ['20241011', '20241018', '20241025', '20241101', '20241108', '20241115', '20241122', '20241129']
Valid strikes for DPZ: [210.0, 220.0, 230.0, 240.0, 250.0, 260.0, 270.0, 280.0, 285.0, 290.0, 295.0, 300.0, 305.0, 310.0, 315.0, 320.0, 325.0, 330.0, 335.0, 340.0, 345.0, 350.0, 355.0, 360.0, 365.0, 370.0, 375.0, 380.0, 385.0, 390.0, 392.5, 395.0, 397.5, 400.0, 402.5, 405.0, 407.5, 410.0, 412.5, 415.0, 417.5, 420.0, 422.5, 425.0, 427.5, 430.0, 432.5, 435.0, 437.5, 440.0, 445.0, 450.0, 455.0, 460.0, 465.0, 470.0, 475.0, 480.0, 485.0, 490.0, 495.0, 500.0, 505.0, 510.0, 515.0, 520.0, 530.0, 540.0, 550.0, 560.0, 570.0, 580.0, 590.0, 600.0, 610.0]


Unknown contract: Option(symbol='DPZ', lastTradeDateOrContractMonth='20241011', strike=210.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='DPZ', lastTradeDateOrContractMonth='20241011', strike=220.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='DPZ', lastTradeDateOrContractMonth='20241011', strike=230.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='DPZ', lastTradeDateOrContractMonth='20241011', strike=240.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='DPZ', lastTradeDateOrContractMonth='20241011', strike=250.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='DPZ', lastTradeDateOrContractMonth='20241011', strike=260.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='DPZ', lastTradeDateOrContractMonth='20241018', strike=210.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='DPZ', lastTradeDateOrContractMonth='20241018', strike=220.0, right='P', exchange='SMART')
Unknown contract

Qualified option contracts for DPZ
Market data received for DPZ
DataFrame created for DPZ:
  Symbol  StockPrice  Strike  Expiration  DTE  Bid  BidSize   Ask  AskSize  \
0    DPZ      408.11   270.0  2024-10-11    1 -1.0      0.0  0.05    320.0   
1    DPZ      408.11   280.0  2024-10-11    1 -1.0      0.0  0.05    280.0   
2    DPZ      408.11   285.0  2024-10-11    1 -1.0      0.0  0.05    281.0   
3    DPZ      408.11   290.0  2024-10-11    1 -1.0      0.0  0.05    281.0   
4    DPZ      408.11   295.0  2024-10-11    1 -1.0      0.0  0.05    286.0   

                                            Contract  
0  Option(conId=726232765, symbol='DPZ', lastTrad...  
1  Option(conId=725859451, symbol='DPZ', lastTrad...  
2  Option(conId=732791603, symbol='DPZ', lastTrad...  
3  Option(conId=725859494, symbol='DPZ', lastTrad...  
4  Option(conId=732791666, symbol='DPZ', lastTrad...  
Filtered options DataFrame for DPZ:
    Symbol  StockPrice  Strike  Expiration  DTE  Bid  BidSize  Ask  AskSiz

,Symbol,StockPrice,Strike,Expiration,DTE,Bid,BidSize,Ask,AskSize,Contract,option_premium,annualized_premium,safety_margin,safety_margin%
309,DPZ,408.11,380.0,2024-11-15,36,4.0,292.0,4.4,321.0,"Option(conId=729830490, symbol='DPZ', lastTrad...",0.010526,0.106725,28.11,0.068878
348,DPZ,408.11,380.0,2024-11-22,43,4.7,51.0,9.2,262.0,"Option(conId=733214874, symbol='DPZ', lastTrad...",0.012368,0.104988,28.11,0.068878
392,DPZ,408.11,385.0,2024-11-29,50,5.7,137.0,8.2,40.0,"Option(conId=734520951, symbol='DPZ', lastTrad...",0.014805,0.108078,23.11,0.056627
264,DPZ,408.11,385.0,2024-11-08,29,3.9,108.0,4.6,150.0,"Option(conId=733212146, symbol='DPZ', lastTrad...",0.010130,0.127497,23.11,0.056627
349,DPZ,408.11,385.0,2024-11-22,43,5.6,61.0,6.5,125.0,"Option(conId=733214932, symbol='DPZ', lastTrad...",0.014545,0.123467,23.11,0.056627
310,DPZ,408.11,390.0,2024-11-15,36,6.1,216.0,6.6,335.0,"Option(conId=729830590, symbol='DPZ', lastTrad...",0.015641,0.158583,18.11,0.044375
350,DPZ,408.11,390.0,2024-11-22,43,6.9,121.0,7.8,151.0,"Option(conId=733214989, symbol='DPZ', lastTrad...",0.017692,0.150179,18.11,0.044375
393,DPZ,408.11,390.0,2024-11-29,50,6.8,125.0,11.8,99.0,"Option(conId=734521002, symbol='DPZ', lastTrad...",0.017436,0.127282,18.11,0.044375
265,DPZ,408.11,390.0,2024-11-08,29,4.9,151.0,5.6,49.0,"Option(conId=731884277, symbol='DPZ', lastTrad...",0.012564,0.158134,18.11,0.044375
394,DPZ,408.11,395.0,2024-11-29,50,8.2,148.0,12.4,114.0,"Option(conId=734521070, symbol='DPZ', lastTrad...",0.020759,0.151544,13.11,0.032124


In [320]:
df

,Symbol,StockPrice,Strike,Expiration,DTE,Bid,BidSize,Ask,AskSize,Contract,option_premium,annualized_premium,safety_margin,safety_margin%
309,DPZ,408.11,380.0,2024-11-15,36,4.0,292.0,4.4,321.0,"Option(conId=729830490, symbol='DPZ', lastTrad...",0.010526,0.106725,28.11,0.068878
348,DPZ,408.11,380.0,2024-11-22,43,4.7,51.0,9.2,262.0,"Option(conId=733214874, symbol='DPZ', lastTrad...",0.012368,0.104988,28.11,0.068878
392,DPZ,408.11,385.0,2024-11-29,50,5.7,137.0,8.2,40.0,"Option(conId=734520951, symbol='DPZ', lastTrad...",0.014805,0.108078,23.11,0.056627
264,DPZ,408.11,385.0,2024-11-08,29,3.9,108.0,4.6,150.0,"Option(conId=733212146, symbol='DPZ', lastTrad...",0.010130,0.127497,23.11,0.056627
349,DPZ,408.11,385.0,2024-11-22,43,5.6,61.0,6.5,125.0,"Option(conId=733214932, symbol='DPZ', lastTrad...",0.014545,0.123467,23.11,0.056627
310,DPZ,408.11,390.0,2024-11-15,36,6.1,216.0,6.6,335.0,"Option(conId=729830590, symbol='DPZ', lastTrad...",0.015641,0.158583,18.11,0.044375
350,DPZ,408.11,390.0,2024-11-22,43,6.9,121.0,7.8,151.0,"Option(conId=733214989, symbol='DPZ', lastTrad...",0.017692,0.150179,18.11,0.044375
393,DPZ,408.11,390.0,2024-11-29,50,6.8,125.0,11.8,99.0,"Option(conId=734521002, symbol='DPZ', lastTrad...",0.017436,0.127282,18.11,0.044375
265,DPZ,408.11,390.0,2024-11-08,29,4.9,151.0,5.6,49.0,"Option(conId=731884277, symbol='DPZ', lastTrad...",0.012564,0.158134,18.11,0.044375
394,DPZ,408.11,395.0,2024-11-29,50,8.2,148.0,12.4,114.0,"Option(conId=734521070, symbol='DPZ', lastTrad...",0.020759,0.151544,13.11,0.032124


In [43]:
async def process_get_filtered_put_options(ib, symbol,min_dte=0, max_dte=60, filtered=False,strike_range_percent = 0.25):
    
    # Get the stock contract
    stock = Stock(symbol, 'SMART', 'USD')
    await ib.qualifyContractsAsync(stock)
    
    # Get the stock price
    [ticker] = await ib.reqTickersAsync(stock)

    stock_price = ticker.marketPrice() if ticker.marketPrice() is not None else ticker.close
    
    # Calculate strike range
    lower_strike = stock_price * (1-strike_range_percent)  
    upper_strike = stock_price * (1+strike_range_percent) 
    
    # Get option chains
    chains = await ib.reqSecDefOptParamsAsync(stock.symbol, '', stock.secType, stock.conId)
    
    # Get the chain with exchange 'SMART'
    chain = next((c for c in chains if c.exchange == 'SMART'), None)
    if not chain:
        print(f"No SMART chain found for {symbol}")
        return None, None
    
    # Get current date
    today = datetime.now().date()
    
    # Filter expirations and strikes
    valid_expirations = [exp for exp in chain.expirations 
                         if (datetime.strptime(exp, '%Y%m%d').date() - today).days >= min_dte and (datetime.strptime(exp, '%Y%m%d').date() - today).days <= max_dte]
    valid_strikes = [strike for strike in chain.strikes 
                     if lower_strike <= strike <= upper_strike]
    
    # Create option contracts
    contracts = [Option(symbol, exp, strike, 'P', 'SMART') 
                 for exp in valid_expirations 
                 for strike in valid_strikes]
    
    # Qualify the contracts
    contracts = await ib.qualifyContractsAsync(*contracts)
    
    # Request market data
    tickers = await ib.reqTickersAsync(*contracts)
    
    # Create DataFrame
    data = []
    for ticker in tickers:
        contract = ticker.contract
        expiration = datetime.strptime(contract.lastTradeDateOrContractMonth, '%Y%m%d').date()
        dte = (expiration - today).days
        data.append({
            'Symbol': symbol,
            'StockPrice': stock_price,
            'Strike': contract.strike,
            'Expiration': expiration,
            'DTE': dte,
            'Bid': ticker.bid,
            'BidSize': ticker.bidSize,
            'Ask': ticker.ask,
            'AskSize': ticker.askSize,
            'Contract': contract})
    
    df = pd.DataFrame(data)
    df = df.sort_values(['Expiration', 'Strike'])

    if filtered:     
        options_df = df.copy()
        options_df['option_premium'] = options_df['Bid'] / options_df['Strike']
        options_df['annualized_premium'] = options_df['option_premium'] * (365 / options_df['DTE'])
        options_df['safety_margin'] = stock_price - options_df['Strike']
        options_df = options_df[(options_df['option_premium'] > 0.01) & (options_df['annualized_premium'] > 0.1)]
        options_df['safety_margin%'] = options_df['safety_margin'] / options_df['StockPrice']
        
        # Sort the DataFrame after creating all columns
        options_df = options_df.sort_values('safety_margin%', ascending=False)
        return options_df.head(10)
    else:
        return df

async def get_filtered_put_options(ib, symbols,min_dte = 0,max_dte = 60, unfiltered=False,strike_range_percent=0.25):
    if not isinstance(symbols, list):
        symbols = [symbols]
        
    tasks = [process_get_filtered_put_options(ib, symbol, min_dte, max_dte, unfiltered,strike_range_percent) for symbol in symbols]
    results = await asyncio.gather(*tasks)

    all_options = []
    if not unfiltered:
        for options_df, _ in results:
            if options_df is not None and not options_df.empty:
                all_options.append(options_df)
    else:
        all_options = results
    if all_options:
        return pd.concat(all_options, ignore_index=True)
    else:
        return pd.DataFrame()

,Symbol,StockPrice,Strike,Expiration,DTE,Bid,BidSize,Ask,AskSize,Contract,premium,annualized_premium
0,SPY,572.485,550.0,2024-10-14,10,0.71,1970.0,0.72,246.0,"Option(conId=732657656, symbol='SPY', lastTrad...",0.001309,0.047782
1,SPY,572.485,555.0,2024-10-14,10,1.02,115.0,1.03,871.0,"Option(conId=732826599, symbol='SPY', lastTrad...",0.001856,0.067739
2,SPY,572.485,560.0,2024-10-14,10,1.50,198.0,1.52,1040.0,"Option(conId=732657672, symbol='SPY', lastTrad...",0.002714,0.099071
3,SPY,572.485,563.0,2024-10-14,10,1.91,477.0,1.93,263.0,"Option(conId=732657701, symbol='SPY', lastTrad...",0.003428,0.125124
4,SPY,572.485,564.0,2024-10-14,10,2.08,43.0,2.10,556.0,"Option(conId=732657731, symbol='SPY', lastTrad...",0.003723,0.135904
...,...,...,...,...,...,...,...,...,...,...,...,...
649,SPY,572.485,645.0,2024-11-29,56,71.59,15.0,72.52,15.0,"Option(conId=709102543, symbol='SPY', lastTrad...",0.112434,0.732829
650,SPY,572.485,650.0,2024-11-29,56,76.58,15.0,77.52,15.0,"Option(conId=715990400, symbol='SPY', lastTrad...",0.119262,0.777330
651,SPY,572.485,655.0,2024-11-29,56,81.58,15.0,82.52,15.0,"Option(conId=715990427, symbol='SPY', lastTrad...",0.125985,0.821150
652,SPY,572.485,660.0,2024-11-29,56,86.58,15.0,87.52,15.0,"Option(conId=715990460, symbol='SPY', lastTrad...",0.132606,0.864307


In [344]:
async def process_get_filtered_put_options(ib, symbol, min_dte=0, max_dte=60, filtered=False, strike_range_percent=0.25):
    ib.reqMarketDataType(4)
    try:
        # Get the stock contract
        stock = Stock(symbol, 'SMART', 'USD')
        await ib.qualifyContractsAsync(stock)
        
        # Get the stock price
        [ticker] = await ib.reqTickersAsync(stock)
        stock_price = ticker.marketPrice() if ticker.marketPrice() is not None else ticker.close
        
        if math.isnan(stock_price) or stock_price is None:
            print(f"Unable to get valid stock price for {symbol}")
            return None

        # Calculate strike range
        lower_strike = stock_price * (1 - strike_range_percent)  
        upper_strike = stock_price * (1 + strike_range_percent) 
        
        # Get option chains
        chains = await ib.reqSecDefOptParamsAsync(stock.symbol, '', stock.secType, stock.conId)
        
        # Get the chain with exchange 'SMART'
        chain = next((c for c in chains if c.exchange == 'SMART'), None)
        if not chain:
            print(f"No SMART chain found for {symbol}")
            return None
        
        # Get current date
        today = datetime.now().date()
        
        # Filter expirations and strikes
        valid_expirations = [exp for exp in chain.expirations 
                             if min_dte <= (datetime.strptime(exp, '%Y%m%d').date() - today).days <= max_dte]
        valid_strikes = [strike for strike in chain.strikes 
                         if lower_strike <= strike <= upper_strike]
        
        if not valid_expirations or not valid_strikes:
            print(f"No valid expirations or strikes found for {symbol}")
            return None

        # Create option contracts
        contracts = [Option(symbol, exp, strike, 'P', 'SMART') 
                     for exp in valid_expirations 
                     for strike in valid_strikes]
        
        # Qualify the contracts
        contracts = await ib.qualifyContractsAsync(*contracts)
        
        # Request market data
        tickers = await ib.reqTickersAsync(*contracts)
        
        # Create DataFrame
        data = []
        for ticker in tickers:
            contract = ticker.contract
            expiration = datetime.strptime(contract.lastTradeDateOrContractMonth, '%Y%m%d').date()
            dte = (expiration - today).days
            data.append({
                'Symbol': symbol,
                'StockPrice': stock_price,
                'Strike': contract.strike,
                'Expiration': expiration,
                'DTE': dte,
                'Bid': ticker.bid,
                'BidSize': ticker.bidSize,
                'Ask': ticker.ask,
                'AskSize': ticker.askSize,
                'Last': ticker.last,
                'Contract': contract})
        
        df = pd.DataFrame(data)
        if df.empty:
            print(f"No option data available for {symbol}")
            return None

        df = df.sort_values(['Expiration', 'Strike'])

        if filtered:     
            options_df = df.copy()
            options_df['option_premium'] = options_df['Bid'] / options_df['Strike']
            options_df['annualized_premium'] = options_df['option_premium'] * (365 / options_df['DTE'])
            options_df['safety_margin'] = stock_price - options_df['Strike']
            options_df = options_df[(options_df['option_premium'] > 0.01) & (options_df['annualized_premium'] > 0.1)]
            options_df['safety_margin%'] = options_df['safety_margin'] / options_df['StockPrice']
            
            # Sort the DataFrame after creating all columns
            options_df = options_df.sort_values('safety_margin%', ascending=False)
            return options_df.head(10)
        else:
            return df
    except Exception as e:
        print(f"Error processing {symbol}: {str(e)}")
        return None

async def get_filtered_put_options(ib, symbols, min_dte=0, max_dte=60, filtered=False, strike_range_percent=0.25):
    if not isinstance(symbols, list):
        symbols = [symbols]
        
    tasks = [process_get_filtered_put_options(ib, symbol, min_dte, max_dte, filtered, strike_range_percent) for symbol in symbols]
    results = await asyncio.gather(*tasks)

    all_options = [df for df in results if df is not None and not df.empty]
    
    if all_options:
        return pd.concat(all_options, ignore_index=True)
    else:
        print("No valid data for any symbols")
        return pd.DataFrame()

In [345]:
symbols

['APLD', 'AZZ', 'BSET', 'DAL', 'DPZ', 'ETWO', 'NEOG', 'RELL', 'THTX', 'TLRY']

In [346]:
df = await get_filtered_put_options(ib, symbols=symbols, filtered=False)

Unknown contract: Option(symbol='APLD', lastTradeDateOrContractMonth='20241115', strike=5.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='APLD', lastTradeDateOrContractMonth='20241115', strike=6.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='APLD', lastTradeDateOrContractMonth='20241115', strike=7.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='APLD', lastTradeDateOrContractMonth='20241115', strike=8.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241011', strike=52.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241011', strike=57.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241011', strike=62.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='DAL', lastTradeDateOrContractMonth='20241011', strike=62.5, right='P', exchange='SMART')
Unknown contract: Option

No valid expirations or strikes found for THTX


In [347]:
df

,Symbol,StockPrice,Strike,Expiration,DTE,Bid,BidSize,Ask,AskSize,Last,Contract
0,APLD,6.92,5.5,2024-10-11,1,-1.0,0.0,-1.0,0.0,0.05,"Option(conId=725834940, symbol='APLD', lastTra..."
1,APLD,6.92,6.0,2024-10-11,1,-1.0,0.0,-1.0,0.0,0.05,"Option(conId=725835091, symbol='APLD', lastTra..."
2,APLD,6.92,6.5,2024-10-11,1,-1.0,0.0,-1.0,0.0,0.10,"Option(conId=725835041, symbol='APLD', lastTra..."
3,APLD,6.92,7.0,2024-10-11,1,-1.0,0.0,-1.0,0.0,0.20,"Option(conId=725835205, symbol='APLD', lastTra..."
4,APLD,6.92,7.5,2024-10-11,1,-1.0,0.0,-1.0,0.0,0.58,"Option(conId=725835140, symbol='APLD', lastTra..."
...,...,...,...,...,...,...,...,...,...,...,...
611,TLRY,1.58,1.5,2024-11-01,22,-1.0,0.0,-1.0,0.0,0.09,"Option(conId=728308367, symbol='TLRY', lastTra..."
612,TLRY,1.58,1.5,2024-11-08,29,-1.0,0.0,-1.0,0.0,0.09,"Option(conId=731949586, symbol='TLRY', lastTra..."
613,TLRY,1.58,1.5,2024-11-15,36,-1.0,0.0,-1.0,0.0,0.11,"Option(conId=701034284, symbol='TLRY', lastTra..."
614,TLRY,1.58,1.5,2024-11-22,43,-1.0,0.0,-1.0,0.0,0.18,"Option(conId=733319128, symbol='TLRY', lastTra..."


In [348]:
options_df = df.copy()
options_df['option_premium'] = options_df['Last'] / options_df['Strike']
options_df['annualized_premium'] = options_df['option_premium'] * (365 / options_df['DTE'])
options_df['safety_margin%'] = (options_df['StockPrice'] - options_df['Strike']) / options_df['StockPrice']
options_df = options_df[(options_df['option_premium'] > 0.01) & (options_df['annualized_premium'] > 0.1)]

# # Sort the DataFrame after creating all columns
# options_df = options_df.sort_values('safety_margin%', ascending=False)
options_df.sort_values('option_premium', ascending=False)

,Symbol,StockPrice,Strike,Expiration,DTE,Bid,BidSize,Ask,AskSize,Last,Contract,option_premium,annualized_premium,safety_margin%
33,APLD,6.920,8.0,2024-11-08,29,-1.0,0.0,-1.0,0.0,1.80,"Option(conId=731843459, symbol='APLD', lastTra...",0.225000,2.831897,-0.156069
37,APLD,6.920,8.0,2024-11-15,36,-1.0,0.0,-1.0,0.0,1.67,"Option(conId=727329824, symbol='APLD', lastTra...",0.208750,2.116493,-0.156069
6,APLD,6.920,8.5,2024-10-11,1,-1.0,0.0,-1.0,0.0,1.65,"Option(conId=725835257, symbol='APLD', lastTra...",0.194118,70.852941,-0.228324
26,APLD,6.920,8.0,2024-11-01,22,-1.0,0.0,-1.0,0.0,1.53,"Option(conId=728189388, symbol='APLD', lastTra...",0.191250,3.173011,-0.156069
25,APLD,6.920,7.5,2024-11-01,22,-1.0,0.0,-1.0,0.0,1.35,"Option(conId=728189162, symbol='APLD', lastTra...",0.180000,2.986364,-0.083815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,DPZ,408.750,405.0,2024-10-18,8,-1.0,0.0,-1.0,0.0,4.60,"Option(conId=730710986, symbol='DPZ', lastTrad...",0.011358,0.518210,0.009174
55,AZZ,76.275,75.0,2024-10-18,8,-1.0,0.0,-1.0,0.0,0.85,"Option(conId=722807032, symbol='AZZ', lastTrad...",0.011333,0.517083,0.016716
126,DAL,50.250,49.0,2024-10-18,8,-1.0,0.0,-1.0,0.0,0.53,"Option(conId=730715721, symbol='DAL', lastTrad...",0.010816,0.493495,0.024876
61,AZZ,76.275,65.0,2024-11-15,36,-1.0,0.0,-1.0,0.0,0.70,"Option(conId=690607768, symbol='AZZ', lastTrad...",0.010769,0.109188,0.147820


In [352]:
o = options_df[['Symbol', 'StockPrice', 'Strike', 'Expiration', 'DTE', 'Last','option_premium','annualized_premium','safety_margin%']]
o = o.rename(columns = {'option_premium':'Premium','annualized_premium':'Annualized Premium','safety_margin%':'Safety Margin'})
o

,Symbol,StockPrice,Strike,Expiration,DTE,Last,Premium,Annualized Premium,Safety Margin
2,APLD,6.92,6.5,2024-10-11,1,0.10,0.015385,5.615385,0.060694
3,APLD,6.92,7.0,2024-10-11,1,0.20,0.028571,10.428571,-0.011561
4,APLD,6.92,7.5,2024-10-11,1,0.58,0.077333,28.226667,-0.083815
5,APLD,6.92,8.0,2024-10-11,1,1.15,0.143750,52.468750,-0.156069
6,APLD,6.92,8.5,2024-10-11,1,1.65,0.194118,70.852941,-0.228324
...,...,...,...,...,...,...,...,...,...
610,TLRY,1.58,1.5,2024-10-25,15,0.05,0.033333,0.811111,0.050633
611,TLRY,1.58,1.5,2024-11-01,22,0.09,0.060000,0.995455,0.050633
612,TLRY,1.58,1.5,2024-11-08,29,0.09,0.060000,0.755172,0.050633
613,TLRY,1.58,1.5,2024-11-15,36,0.11,0.073333,0.743519,0.050633


In [353]:
o[o['Safety Margin'] > 0.05].sort_values('Annualized Premium', ascending=False)

,Symbol,StockPrice,Strike,Expiration,DTE,Last,Premium,Annualized Premium,Safety Margin
2,APLD,6.920,6.5,2024-10-11,1,0.10,0.015385,5.615385,0.060694
9,APLD,6.920,6.5,2024-10-18,8,0.30,0.046154,2.105769,0.060694
16,APLD,6.920,6.5,2024-10-25,15,0.50,0.076923,1.871795,0.060694
596,ETWO,3.190,3.0,2024-10-18,8,0.10,0.033333,1.520833,0.059561
23,APLD,6.920,6.5,2024-11-01,22,0.50,0.076923,1.276224,0.060694
22,APLD,6.920,6.0,2024-11-01,22,0.45,0.075000,1.244318,0.132948
609,TLRY,1.580,1.5,2024-10-18,8,0.04,0.026667,1.216667,0.050633
8,APLD,6.920,6.0,2024-10-18,8,0.15,0.025000,1.140625,0.132948
614,TLRY,1.580,1.5,2024-11-22,43,0.18,0.120000,1.018605,0.050633
35,APLD,6.920,6.0,2024-11-15,36,0.60,0.100000,1.013889,0.132948


In [238]:
async def get_filtered_put_options_async(ib, symbol, max_dte=60, safety_margin=0.5):
    # Get the stock contract
    stock = Stock(symbol, 'SMART', 'USD')
    await ib.qualifyContractsAsync(stock)
    
    # Get the stock price
    [ticker] = await ib.reqTickersAsync(stock)
    stock_price = ticker.marketPrice()
    
    if np.isnan(stock_price):
        print(f"Unable to get market price for {symbol}")
        return None, None
    
    # Calculate strike range
    lower_strike = stock_price * (1-safety_margin)  # 15% below
    upper_strike = stock_price * (1+safety_margin)  # 5% above
    
    # Get option chains
    chains = await ib.reqSecDefOptParamsAsync(stock.symbol, '', stock.secType, stock.conId)
    
    # Get the chain with exchange 'SMART'
    chain = next((c for c in chains if c.exchange == 'SMART'), None)
    if not chain:
        print(f"No SMART chain found for {symbol}")
        return None, None
    
    # Get current date
    today = datetime.now().date()
    
    # Filter expirations and strikes
    valid_expirations = [exp for exp in chain.expirations 
                         if (datetime.strptime(exp, '%Y%m%d').date() - today).days <= max_dte]
    valid_strikes = [strike for strike in chain.strikes 
                     if lower_strike <= strike <= upper_strike]
    
    # Create option contracts
    contracts = [Option(symbol, exp, strike, 'P', 'SMART') 
                 for exp in valid_expirations 
                 for strike in valid_strikes]
    
    # Qualify the contracts
    contracts = await ib.qualifyContractsAsync(*contracts)
    
    # Request market data
    tickers = await ib.reqTickersAsync(*contracts)
    
    # Create DataFrame
    data = []
    for ticker in tickers:
        contract = ticker.contract
        expiration = datetime.strptime(contract.lastTradeDateOrContractMonth, '%Y%m%d').date()
        dte = (expiration - today).days
        data.append({
            'Symbol': symbol,
            'Strike': contract.strike,
            'Expiration': expiration,
            'DTE': dte,
            'Bid': ticker.bid,
            'BidSize': ticker.bidSize,
            'Ask': ticker.ask,
            'AskSize': ticker.askSize,
            'Contract': contract
        })
    
    df = pd.DataFrame(data)
    df = df.sort_values(['Expiration', 'Strike'])
    
    options_df = df.copy()
    options_df['option_premium'] = options_df['Bid'] / options_df['Strike']
    options_df['annualized_premium'] = options_df['option_premium'] * (365 / options_df['DTE'])
    options_df['stock_price'] = stock_price
    options_df['safety_margin'] = stock_price - options_df['Strike']
    options_df = options_df[(options_df['option_premium'] > 0.01) & (options_df['annualized_premium'] > 0.1)]
    options_df['safety_margin%'] = options_df['safety_margin'] / options_df['stock_price']
    
    # Sort the DataFrame after creating all columns
    options_df = options_df.sort_values('safety_margin%', ascending=False)
    
    return options_df.head(10), stock_price

async def process_all_symbols(ib, symbols):
    tasks = [get_filtered_put_options_async(ib, symbol) for symbol in symbols]
    results = await asyncio.gather(*tasks)
    
    all_options = []
    for options_df, stock_price in results:
        if options_df is not None and not options_df.empty:
            all_options.append(options_df)
    
    if all_options:
        return pd.concat(all_options, ignore_index=True)
    else:
        return pd.DataFrame()

In [239]:
symbols = ['LW', 'CALM', 'RPM', 'NKE', 'CAG']  # Your list of symbols
result_df = await process_all_symbols(ib, symbols)
    
if not result_df.empty:
    result_df.sort_values('safety_margin%', ascending=False)

Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241004', strike=72.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241004', strike=77.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241004', strike=82.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241011', strike=72.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241011', strike=77.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241011', strike=82.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241025', strike=72.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241025', strike=77.5, right='P', exchange='SMART')
Unknown contract: Option

In [10]:
result_df['decision_ind'] = result_df['safety_margin%'] / result_df['option_premium']
result_df.sort_values('decision_ind', ascending=False)

NameError: name 'result_df' is not defined

In [229]:
options_df['safety_margin%'] = options_df['safety_margin'] / options_df['stock_price']
options_df


,Strike,Expiration,DTE,Bid,BidSize,Ask,AskSize,Contract,option_premium,annualized_premium,stock_price,safety_margin,safety_margin%
97,77.5,2024-11-15,44,0.96,790.0,1.06,2459.0,"Option(conId=719526311, symbol='NKE', lastTrad...",0.012387,0.102757,82.555,5.055,0.061232
87,79.0,2024-11-08,37,0.88,339.0,1.29,137.0,"Option(conId=731946413, symbol='NKE', lastTrad...",0.011139,0.109887,82.555,3.555,0.043062
88,80.0,2024-11-08,37,1.41,167.0,1.56,99.0,"Option(conId=731946473, symbol='NKE', lastTrad...",0.017625,0.173868,82.555,2.555,0.030949
98,80.0,2024-11-15,44,1.66,36.0,1.72,899.0,"Option(conId=705993053, symbol='NKE', lastTrad...",0.020750,0.172131,82.555,2.555,0.030949
60,80.0,2024-10-25,23,0.93,1040.0,1.04,2268.0,"Option(conId=727047636, symbol='NKE', lastTrad...",0.011625,0.184484,82.555,2.555,0.030949
76,80.0,2024-11-01,30,1.10,139.0,1.19,518.0,"Option(conId=728253068, symbol='NKE', lastTrad...",0.013750,0.167292,82.555,2.555,0.030949
77,81.0,2024-11-01,30,1.40,977.0,1.57,1037.0,"Option(conId=728253102, symbol='NKE', lastTrad...",0.017284,0.210288,82.555,1.555,0.018836
61,81.0,2024-10-25,23,1.24,869.0,1.43,4500.0,"Option(conId=727047673, symbol='NKE', lastTrad...",0.015309,0.242941,82.555,1.555,0.018836
44,81.0,2024-10-18,16,1.04,155.0,1.12,1978.0,"Option(conId=730757679, symbol='NKE', lastTrad...",0.012840,0.292901,82.555,1.555,0.018836
89,81.0,2024-11-08,37,1.72,115.0,1.95,175.0,"Option(conId=731946531, symbol='NKE', lastTrad...",0.021235,0.209476,82.555,1.555,0.018836


In [220]:
a, _ = get_filtered_put_options(ib, symbols[0])

In [225]:
a

,Strike,Expiration,DTE,Bid,BidSize,Ask,AskSize,Contract,option_premium,annualized_premium,stock_price,safety_margin
6,60.0,2024-11-15,44,0.95,177.0,1.15,713.0,"Option(conId=729997681, symbol='LW', lastTrade...",0.015833,0.131345,65.67,5.67
2,62.5,2024-10-18,16,0.80,11.0,0.95,551.0,"Option(conId=718404917, symbol='LW', lastTrade...",0.012800,0.292000,65.67,3.17
7,62.5,2024-11-15,44,1.60,39.0,1.95,179.0,"Option(conId=729997734, symbol='LW', lastTrade...",0.025600,0.212364,65.67,3.17
3,65.0,2024-10-18,16,1.65,389.0,1.90,10.0,"Option(conId=684662461, symbol='LW', lastTrade...",0.025385,0.579087,65.67,0.67
8,65.0,2024-11-15,44,2.60,3.0,2.95,1112.0,"Option(conId=729997787, symbol='LW', lastTrade...",0.040000,0.331818,65.67,0.67
4,67.5,2024-10-18,16,2.90,78.0,3.30,18.0,"Option(conId=721751619, symbol='LW', lastTrade...",0.042963,0.980093,65.67,-1.83
9,67.5,2024-11-15,44,3.80,32.0,4.30,3.0,"Option(conId=729997854, symbol='LW', lastTrade...",0.056296,0.467003,65.67,-1.83


In [221]:
b, _ = get_filtered_put_options(ib, symbols[1])

In [226]:
b

,Strike,Expiration,DTE,Bid,BidSize,Ask,AskSize,Contract,option_premium,annualized_premium,stock_price,safety_margin
7,70.0,2024-11-15,44,0.90,305.0,1.25,117.0,"Option(conId=690655193, symbol='CALM', lastTra...",0.012857,0.106656,76.6,6.6
8,72.5,2024-11-15,44,1.50,369.0,2.05,166.0,"Option(conId=695628006, symbol='CALM', lastTra...",0.020690,0.171630,76.6,4.1
3,75.0,2024-10-18,16,1.00,528.0,1.35,356.0,"Option(conId=722829516, symbol='CALM', lastTra...",0.013333,0.304167,76.6,1.6
9,75.0,2024-11-15,44,2.45,202.0,2.90,104.0,"Option(conId=690655266, symbol='CALM', lastTra...",0.032667,0.270985,76.6,1.6
4,77.5,2024-10-18,16,2.15,120.0,2.50,209.0,"Option(conId=722829606, symbol='CALM', lastTra...",0.027742,0.632863,76.6,-0.9
10,77.5,2024-11-15,44,3.50,191.0,4.20,127.0,"Option(conId=719488007, symbol='CALM', lastTra...",0.045161,0.374633,76.6,-0.9
5,80.0,2024-10-18,16,3.40,393.0,4.30,203.0,"Option(conId=722829683, symbol='CALM', lastTra...",0.042500,0.969531,76.6,-3.4
11,80.0,2024-11-15,44,5.10,99.0,5.70,160.0,"Option(conId=690655368, symbol='CALM', lastTra...",0.063750,0.528835,76.6,-3.4


In [222]:
c, _ = get_filtered_put_options(ib, symbols[2])

In [227]:
c

,Strike,Expiration,DTE,Bid,BidSize,Ask,AskSize,Contract,option_premium,annualized_premium,stock_price,safety_margin
7,120.0,2024-11-15,44,2.00,203.0,2.35,73.0,"Option(conId=690977232, symbol='RPM', lastTrad...",0.016667,0.138258,124.52,4.52
3,125.0,2024-10-18,16,2.45,169.0,2.95,140.0,"Option(conId=723235348, symbol='RPM', lastTrad...",0.019600,0.447125,124.52,-0.48
8,125.0,2024-11-15,44,3.80,314.0,4.30,65.0,"Option(conId=690977306, symbol='RPM', lastTrad...",0.030400,0.252182,124.52,-0.48
4,130.0,2024-10-18,16,4.70,212.0,8.30,132.0,"Option(conId=723235423, symbol='RPM', lastTrad...",0.036154,0.824760,124.52,-5.48
9,130.0,2024-11-15,44,6.40,589.0,7.40,71.0,"Option(conId=690977383, symbol='RPM', lastTrad...",0.049231,0.408392,124.52,-5.48


In [223]:
d, _ = get_filtered_put_options(ib, symbols[3])
d

Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241004', strike=72.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241004', strike=77.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241004', strike=82.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241011', strike=72.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241011', strike=77.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241011', strike=82.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241025', strike=72.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='NKE', lastTradeDateOrContractMonth='20241025', strike=77.5, right='P', exchange='SMART')
Unknown contract: Option

,Strike,Expiration,DTE,Bid,BidSize,Ask,AskSize,Contract,option_premium,annualized_premium,stock_price,safety_margin
104,77.5,2024-11-15,44,0.98,1995.0,1.05,898.0,"Option(conId=719526311, symbol='NKE', lastTrad...",0.012645,0.104897,82.13,4.63
92,78.0,2024-11-08,37,0.82,430.0,1.00,1.0,"Option(conId=731946347, symbol='NKE', lastTrad...",0.010513,0.103708,82.13,4.13
93,79.0,2024-11-08,37,0.98,356.0,1.29,121.0,"Option(conId=731946413, symbol='NKE', lastTrad...",0.012405,0.122374,82.13,3.13
105,80.0,2024-11-15,44,1.69,132.0,1.74,214.0,"Option(conId=705993053, symbol='NKE', lastTrad...",0.021125,0.175241,82.13,2.13
64,80.0,2024-10-25,23,0.92,3911.0,1.05,35.0,"Option(conId=727047636, symbol='NKE', lastTrad...",0.011500,0.182500,82.13,2.13
94,80.0,2024-11-08,37,1.35,388.0,1.74,952.0,"Option(conId=731946473, symbol='NKE', lastTrad...",0.016875,0.166470,82.13,2.13
81,80.0,2024-11-01,30,1.12,506.0,1.22,749.0,"Option(conId=728253068, symbol='NKE', lastTrad...",0.014000,0.170333,82.13,2.13
82,81.0,2024-11-01,30,1.46,22.0,1.99,2167.0,"Option(conId=728253102, symbol='NKE', lastTrad...",0.018025,0.219300,82.13,1.13
65,81.0,2024-10-25,23,1.30,1.0,1.49,3228.0,"Option(conId=727047673, symbol='NKE', lastTrad...",0.016049,0.254697,82.13,1.13
47,81.0,2024-10-18,16,1.08,569.0,1.30,6167.0,"Option(conId=730757679, symbol='NKE', lastTrad...",0.013333,0.304167,82.13,1.13


In [224]:
e, _ = get_filtered_put_options(ib, symbols[4])
e

Unknown contract: Option(symbol='CAG', lastTradeDateOrContractMonth='20241025', strike=27.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='CAG', lastTradeDateOrContractMonth='20241025', strike=28.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='CAG', lastTradeDateOrContractMonth='20241025', strike=29.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='CAG', lastTradeDateOrContractMonth='20241025', strike=30.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='CAG', lastTradeDateOrContractMonth='20241025', strike=31.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='CAG', lastTradeDateOrContractMonth='20241101', strike=27.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='CAG', lastTradeDateOrContractMonth='20241101', strike=28.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='CAG', lastTradeDateOrContractMonth='20241101', strike=29.5, right='P', exchange='SMART')
Unknown contract: Option

,Strike,Expiration,DTE,Bid,BidSize,Ask,AskSize,Contract,option_premium,annualized_premium,stock_price,safety_margin
53,28.0,2024-11-15,44,0.30,3752.0,0.40,1898.0,"Option(conId=729765398, symbol='CAG', lastTrad...",0.010714,0.088880,30.13,2.13
42,29.0,2024-11-01,30,0.35,2776.0,0.55,487.0,"Option(conId=728192123, symbol='CAG', lastTrad...",0.012069,0.146839,30.13,1.13
54,29.0,2024-11-15,44,0.55,2480.0,0.65,1499.0,"Option(conId=729765446, symbol='CAG', lastTrad...",0.018966,0.157328,30.13,1.13
48,29.0,2024-11-08,37,0.45,422.0,0.65,67.0,"Option(conId=731851019, symbol='CAG', lastTrad...",0.015517,0.153075,30.13,1.13
28,29.5,2024-10-18,16,0.30,2129.0,0.40,628.0,"Option(conId=730690510, symbol='CAG', lastTrad...",0.010169,0.231992,30.13,0.63
37,30.0,2024-10-25,23,0.50,3474.0,0.65,888.0,"Option(conId=727000154, symbol='CAG', lastTrad...",0.016667,0.264493,30.13,0.13
55,30.0,2024-11-15,44,0.95,1541.0,1.05,1641.0,"Option(conId=729765493, symbol='CAG', lastTrad...",0.031667,0.262689,30.13,0.13
18,30.0,2024-10-11,9,0.40,1520.0,0.50,730.0,"Option(conId=725847510, symbol='CAG', lastTrad...",0.013333,0.540741,30.13,0.13
49,30.0,2024-11-08,37,0.80,417.0,1.05,68.0,"Option(conId=731851069, symbol='CAG', lastTrad...",0.026667,0.263063,30.13,0.13
29,30.0,2024-10-18,16,0.50,1267.0,0.60,3711.0,"Option(conId=722834139, symbol='CAG', lastTrad...",0.016667,0.380208,30.13,0.13


In [200]:
options_df['option_premium'] = options_df['Bid'] / options_df['Strike']
options_df['annualized_premium'] = options_df['option_premium'] * (365 / options_df['DTE'])
options_df['stock_price'] = stock_price
options_df['safety_margin'] = stock_price - options_df['Strike']


